In [2]:
# check if ABG corpus was already downloaded
# if not, download it
if [ ! -f /tmp/Corpus_ABG.csv ]; then 
  wget -q https://raw.githubusercontent.com/SauronGuide/corpusABG/master/Corpus_ABG_Completo_Versao3.csv -O /tmp/Corpus_ABG.csv
fi

Check ABG Corpus file type.

In [3]:
file /tmp/Corpus_ABG.csv

/tmp/Corpus_ABG.csv: UTF-8 Unicode (with BOM) text, with CRLF line terminators


Since it uses CRLF line terminators, it was probably created on Windows. This double line terminator is unnacessary and carries an extra byte (```\r```, carriage return) that might appear as ```^M``` on Linux and create further problem.

The BOM marker is also redundant, since it is possible to infer the endianness by a simple analysis of the data. The BOM marker (in UTF-8) is made of the three inicial bytes ```0xEF```,```0xBB```,```0xBF```.

So let's start by removing both of them.

In [4]:
tail --bytes=+4 /tmp/Corpus_ABG.csv | tr -d '\r' > /tmp/CorpusABG.csv

Unfortunately the ABG corpus has many erros. We will try to fix some of them and leave many behind.

The first error we found is empty lines, that appear as: ```,,,,,,,,,,,,,,,,```.

Lets list them and remove them. We will use grep to find them and the parameter ```-n``` to print the line number, along the match.

In [5]:
grep -n '^,' /tmp/CorpusABG.csv 

20608:,,,,,,,,,,,,,,,,
20747:,,,,,,,,,,,,,,,,
20957:,,,,,,,,,,,,,,,,
48422:,,,,,,,,,,,,,,,,
48764:,,,,,,,,,,,,,,,,
49034:,,,,,,,,,,,,,,,,
49705:,,,,,,,,,,,,,,,,
49838:,,,,,,,,,,,,,,,,
50175:,,,,,,,,,,,,,,,,
51587:,,,,,,,,,,,,,,,,
53205:,,,,,,,,,,,,,,,,
92625:,,   ,,,,,,,,,,,,,,


Now, lets remove those lines. We're using the same command from above and cutting the result to get only the line number. Then we peform a loop on those numbers to create a string with everything we will remove using ```sed```.

In [6]:
LINES=""; 
for i in `grep -n '^,' /tmp/CorpusABG.csv | cut -d: -f1`; do 
  LINES="${LINES}${i}d;"; 
done; 
sed -ie "${LINES}" /tmp/CorpusABG.csv

The corpus has a sequence of commas ```,,,,,``` in the end of each line. We will remove it, since it is unused.

In [7]:
sed -i 's/,,,,,$//g' /tmp/CorpusABG.csv

The fields (columns) in the ABG corpus are listed on the first line.

In [8]:
head -n 1 /tmp/CorpusABG.csv 

ID,PALAVRA,CATMORF,LEMA,TRANSCRICAO,ACENTUACAO,ESTSILABICA,CATACENTUAL,FREQGERAL,FREQORAL,FREQESCRITA,Freq_Nivel


There are 12 fields: ID (1), PALAVRA (2), CATMORF (3), LEMA (4), TRANSCRICAO (5), ACENTUACAO (6), ESTSILABICA (7), CATACENTUAL (8), FREQGERAL (9), FREQORAL (10), FREQESCRITA (11), Freq_Nivel (12).

Another type of error we found in the database is lines that don't have exactly 12 fields (3 have more than 12 and 3 have less than 12). Probably the authors have inserted erroneous commas or have missed some, creating a malformed file. The script bellow present the rows that do not have 12 fields. We also print the line number, so we may once again remove the bad data.

In [9]:
cat /tmp/CorpusABG.csv | sed 's/,\+\s*$//' | awk -F, 'NF!=12{print NR":"$0}'

24089:24093,Jereissati,   F   jereissati,   &je-reJ-sa-Ti*,   &je-reJ-s1-Ty*,   &CV-CVG-CV-CV*,   parox?tona,6,   0o,6,2
35806:35816,Fuentes,   F   Fuentes,   &fE-tes*,   &fE-t4s*,   &CV-CVS*,   ox?tona,3,   0o,3,2
46495:46508,prociss?es,   NP,,   prociss?es,   &pro-si-sOJs*,   &pro-si-s#Js*,   &CCV-CV-CVGS*,   ox?tona,2,2,   0e,2
51341:51359,Josh,   F   Josh,   &jo-Sy*,   &j9-Sy*,   &CV-CV*,   parox?tona,2,   0o,2,2
54571:54607,Juvenile,   NOM,   Juvenile,   &ju-ve-ni-le*,   &ju-ve-n7-ly*,   &CV-CV-CV-CV*,   parox?tona,1,   0o,1,,54607
77325:77959,bulut,   F,   bulut,   &bu-lut*,   &bu-l$t*,   &CV-CVC*,   ox?tona,1,   0o,1,1,,1,   0o,1,1


Lets then remove those lines, just as we did before.

In [10]:
LINES=""; 
for i in `sed 's/,\+\s*$//' < /tmp/Corpus_ABG.csv | awk -F, 'NF!=12{print NR":"$0}' | cut -d: -f1`; do 
  LINES="${LINES}${i}d;"; 
done; 
sed -ie "${LINES}" /tmp/CorpusABG.csv

There are also rows where the FREQGERAL is zero or not a number. 

In [11]:
awk -F, '$9<1{print NR":"$0}' < /tmp/CorpusABG.csv

24086:24093,Jereissati,   F   jereissati,   &je-reJ-sa-Ti*,   &je-reJ-s1-Ty*,   &CV-CVG-CV-CV*,   parox?tona,6,   0o,6,2
35802:35816,Fuentes,   F   Fuentes,   &fE-tes*,   &fE-t4s*,   &CV-CVS*,   ox?tona,3,   0o,3,2
46490:46508,prociss?es,   NP,,   prociss?es,   &pro-si-sOJs*,   &pro-si-s#Js*,   &CCV-CV-CVGS*,   ox?tona,2,2,   0e,2
51329:51359,Josh,   F   Josh,   &jo-Sy*,   &j9-Sy*,   &CV-CV*,   parox?tona,2,   0o,2,2


We're going to remove these data as well.

In [12]:
LINES=""; 
for i in `awk -F, '$9<1{print NR":"$0}' < /tmp/CorpusABG.csv | cut -d: -f1`; do 
  LINES="${LINES}${i}d;"; 
done; 
sed -ie "${LINES}" /tmp/CorpusABG.csv

There are other frequency fields (FREQORAL (10), FREQESCRITA (11), Freq_Nivel (12)) and theit values should be numeric and positive. But many have non valid values.

In [13]:
awk -F, '($10<1||$11<1||$12<1){print NR":"$0}' < /tmp/CorpusABG.csv | wc -l

71043


This number represents a large amount of the total dada. We will leave it be.

In [14]:
TOTAL=$(wc -l /tmp/CorpusABG.csv | cut -d' ' -f1)
COUNTNN=$(awk -F, '($10<1||$11<1||$12<1){print NR":"$0}' < /tmp/CorpusABG.csv | wc -l)
echo "$COUNTNN/$TOTAL" | bc -l

.76726931052358735095


The morphological category (column 3) and stress category (column 8) are categorical variables, they might have values in a finite set and their value assigning each sample to a different group. Lets check the values used in the corpus.


Morphological category (column 3):

In [15]:
awk -F',[ ]*' '{print $3}' /tmp/CorpusABG.csv | sort | uniq -c | sort -rn

  36563 NOM
  32776 V
  11681 ADJ
   5067 F
   2188 V+P
   1456 C
   1061 ADV
    717 
    430 G
    184 P
    179 I
     80 PREP+P
     55 CONJ
     42 NUM
     39 PREP
     38 PREP+DET
     27 DET
      5 PREP+ADV
      1 V+p
      1 FNOM
      1 f
      1 CATMORF


We see there 717 with no morphological category assigned, there is one "V+p" that might be "V+P" and a "f" that might be "F". We might easily correct those two.

In [16]:
awk -F',[ ]*' 'BEGIN{OFS=", "} ($3=="f"){$3="F"} ($3=="V+p"){$3="V+P"} {print}' /tmp/CorpusABG.csv > /tmp/tmpABG
mv /tmp/tmpABG /tmp/CorpusABG.csv

The list of the 717 of entries with empty morphological category (column 3) is given bellow:

In [17]:
awk -F',[ ]*' '($3==""){print $1,$2}' /tmp/CorpusABG.csv | column

143 legal		395 crian?as		634 logo
149 segundo		396 pol?cia		635 estados
150 vida		397 sala		636 velho
151 outros		398 p?blico		637 m?dico
152 nunca		399 enquanto		638 a??o
153 foram		400 nosso		639 mal
154 ?poca		401 junto		640 votos
155 ia			402 sul			641 sociedade
156 S?o			403 falava		642 alunos
157 disse		404 quarto		643 relacionadas
158 est?o		405 situa??o		644 pequena
159 fui			406 talvez		645 aquelas
160 tipo		407 social		646 debate
161 ir			408 conhece		647 gostoso
162 quer		409 seguran?a		648 trabalhando
163 dentro		410 umas		649 antigamente
164 pessoal		411 idade		650 quiser
165 ficou		412 m?s			651 dilma
166 d?			413 dados		652 papel
167 ver			414 for			653 l?ngua
168 dizer		415 podem		654 equipe
169 falei		416 Mas			655 saiu
170 maior		417 bairros		656 seguinte
171 seus		418 importante		657 cinema
172 entendeu		419 filha		658 pensar
173 teve		420 pouquinho		659 ruim
174 jeito		421 conhe?o		660 eleitoral
175 tanto		422 curso		661 cidades
176 essas		423 processo		662 ideia
17

391 professor		630 entrevista		869 obrigada
392 viol?ncia		631 condi??es		870 Petrobras
393 viu			632 certeza		871 guerra
394 zona		633 baixo		872 poucos


Stress category (column 8):

In [18]:
awk -F',[ ]*' '{print $8}' /tmp/CorpusABG.csv | sort | uniq -c | sort -rn

  55262 parox?tona
  21694 ox?tona
   6800 paroxitona
   3470 oxitona
   3392 proparox?tona
   1530 mono
    396 proparoxitona
     40 4
      2 PARox?tona
      2 ox?tono
      1 quatro
      1 parox?ton
      1 parox?ona
      1 CATACENTUAL


There are 40 entries with value 4 for stress category, one entry with value 'quatro' (might be the same as 4) and there are many mistyped names which we might correct with a simple substitution.

In [19]:
# fix stress category
sed -i 's/ox?tona/oxitona/g' /tmp/CorpusABG.csv
sed -i 's/ox?tono/oxitona/g' /tmp/CorpusABG.csv
sed -i 's/PARoxitona/paroxitona/g' /tmp/CorpusABG.csv
sed -i 's/parox?ona/paroxitona/g' /tmp/CorpusABG.csv
sed -i 's/parox?ton/paroxitona/g' /tmp/CorpusABG.csv

Now, lets check some of those entries with value 4 in stress category.

In [20]:
awk -F',[ ]*' '($8==4){print}' /tmp/CorpusABG.csv | head -n 5

719,t?cnico,,   t?cnico,   &t5-ky-ni-ko*,   &t5-ky-ny-kw*,   &CV-CV-CV-CV*,4,500,151,349,3
1485,t?cnica,   NOM,   t?cnica,   &t5-ky-ni-ka*,   &t5-ky-ny-k@*,   &CV-CV-CV-CV*,4,231,70,161,3
2666,t?cnicos,   ADJ,   t?cnicos,   &t5-ky-ni-kos*,   &t5-ky-ny-kws*,   &CV-CV-CV-CVS*,4,121,13,108,3
4111,t?cnicas,   ADJ,   t?cnicas,   &t5-ky-ni-kas*,   &t5-ky-ny-k@s*,   &CV-CV-CV-CVC*,4,73,11,62,2
5060,d?ficit,   NOM,   d?ficit,   &d5-fi-si-ty*,   &d5-fy-sy-ty*,   &CV-CV-CV-CV*,4,56,3,53,2


And the complete list of words with value 4 (or 'quatro') in stress category is:

In [21]:
awk -F',[ ]*' '($8==4||$8=="quatro"){print $1,$2}' /tmp/CorpusABG.csv | column

719 t?cnico		31949 Polit?cnica	56961 antiss?ptico
1485 t?cnica		34472 eletrot?cnico	57174 antiss?pticos
2666 t?cnicos		34935 ?tnicas		57581 apocal?pticas
4111 t?cnicas		41512 aut?ctones	59961 c?psula
5060 d?ficit		42759 eletrot?cnica	64142 el?ptico
13979 l?xico		43225 el?ptica		64157 epil?ptica
16379 polit?cnica	44164 ex-t?cnico	64278 epil?pticos
16904 ?tnica		46546 logar?tmica	72143 lepid?pteros
19590 inc?gnita		50151 pan?ptico		74419 g?ngsteres
21275 ?tnicos		51463 r?tmico		77274 multi?tnico
23465 ?tnico		51627 sociot?cnica	83538 d?couvert
26420 inc?gnito		56107 h?bitat		84891 pirot?cnicos
28814 apocal?ptica	56835 anal?pticos	87837 Polit?cnico
29173 c?psulas		56954 antiss?ptica


It seems all of them are proparoxytone ('proparoxitona') in fact. So lets correct them.

In [22]:
awk -F',[ ]*' 'BEGIN{OFS=", "} ($8==4||$8=="quatro"){$8="proparoxitona"} {print}' /tmp/CorpusABG.csv > /tmp/tmpABG
mv /tmp/tmpABG /tmp/CorpusABG.csv

Now lets make an histogram for the stress category.

In [23]:
awk -F',[ ]*' '{print $8}' /tmp/CorpusABG.csv | sort | uniq -c | sort -rn | 
  head -n -1 | nl| 
  gnuplot -e "set terminal png; set output 'images/stress_category.png'; set xlabel 'categoria acentual'; set ylabel 'frequencia'; set style fill solid; set boxwidth 1; set title 'corpus abg'; set xtics rotate by 45 right; plot '/dev/stdin' using 1:2:xtic(3) with boxes notitle"

![](images/stress_category.png)

Every row should have 12 fields, but there are two rows with a different number. One with 16 and another with 17.

In [24]:
awk -F'[ ]*,[ ]*' 'BEGIN{OFS=","} {print NF}' /tmp/CorpusABG.csv | sort | uniq -c

  92590 12
      1 16
      1 17


These rows are presented bellow:

In [25]:
awk -F'[ ]*,[ ]*' 'BEGIN{OFS=","} (NF==16||NF==17){print}' /tmp/CorpusABG.csv

54607,Juvenile,   NOM,   Juvenile,   &ju-ve-ni-le*,   &ju-ve-n7-ly*,   &CV-CV-CV-CV*,   paroxitona,1,   0o,1,,54607,,,
77959,bulut,   F,   bulut,   &bu-lut*,   &bu-l$t*,   &CV-CVC*,   oxitona,1,   0o,1,1,,1,   0o,1,1


Lets correct those lines. Squeezing repeates of commas (,) and removing comma (,) in the end of the line. After lets print only the 12 first fields, since the line for **bulut** has a repeaded values of ones in the end.

In [26]:
awk -F'[ ]*,[ ]*' 'BEGIN{OFS=","} (NF>12){system("echo \""$0"\" | tr -s ',' | sed 's/,$//'")} (NF==12){print}' /tmp/CorpusABG.csv > /tmp/abgtmp
mv /tmp/abgtmp /tmp/CorpusABG.csv 
awk -F'[ ]*,[ ]*' 'BEGIN{OFS=","} (NF==12){print} (NF>12){print $1,$2,$3,$4,$5,$6,$7,$8,$9,$10,$11,$12}' /tmp/CorpusABG.csv > /tmp/abgtmp
mv /tmp/abgtmp /tmp/CorpusABG.csv 

Fields 9, 10, 11 and 12 should be numeric, since they represent frequencies. Lets check how many are not numeric.

In [27]:
tail -n +2 /tmp/CorpusABG.csv | awk -F'[ ]*,[ ]*' 'BEGIN{OFS=","} (!($9~/^[0-9]+$/)){count9++} (!($10~/^[0-9]+$/)){count10++} (!($11~/^[0-9]+$/)){count11++} (!($12~/^[0-9]+$/)){count12++} END{printf "number of non-numeric values in fields: 9:%d, 10:%d, 11:%d, 12:%d\n",count9,count10,count11,count12}'

number of non-numeric values in fields: 9:0, 10:56110, 11:14933, 12:0


Lets check what is written in these fields that were supposed to be numeric.

In [28]:
tail -n +2 /tmp/CorpusABG.csv | awk -F'[ ]*,[ ]*' 'BEGIN{OFS=","} (!($9~/^[0-9]+$/)){print $9} (!($10~/^[0-9]+$/)){print $10} (!($11~/^[0-9]+$/)){print $11} (!($12~/^[0-9]+$/)){print $12}' | sort | uniq -c

  14933 0e
  56109 0o
      1 o


It seems we may replace those values (0e, 0o and o) by 0 (zero). So lets do it.

In [29]:
awk -F'[ ]*,[ ]*' 'BEGIN{OFS=","} (NR>1&&!($9~/^[0-9]+$/)){$9=0} (NR>1&&!($10~/^[0-9]+$/)){$10=0} (NR>1&&!($11~/^[0-9]+$/)){$11=0} (NR>1&&!($12~/^[0-9]+$/)){$12=0} {print}' /tmp/CorpusABG.csv > /tmp/tmpabg 
mv /tmp/tmpabg /tmp/CorpusABG.csv 

Many entries have ```?``` in the word (column 2) or lemma (column 5) transcription. The amount is:

In [30]:
awk -F',[ ]*' '($2~/?/)||($4~/?/){print $2, $4, $5}' /tmp/CorpusABG.csv | wc -l

18007


and some examples are given bellow:

In [31]:
awk -F',[ ]*' '($2~/?/)||($4~/?/){print $2, $4, $5}' /tmp/CorpusABG.csv | head

l? l? &l1*
tamb?m tamb?m &tA-b6*
est? est? &es-t1*
s?o s?o &sAW*
j? j? &j1*
s? s? &s!*
? ? a&a*
at? at? &a-t5*
?s ?s &as*
m?e m?e &mA-e*


In the ABG Corpus repository there is a file ```Corpus_Tag_Freq_Trans.txt``` (probably a intermediary file) which has some data that might be used to fix those ```?``` in the corpus file. 

Lets first download this file:

In [32]:
if [ ! -f /tmp/Acentuador.zip ]; then 
  wget -q https://github.com/SauronGuide/corpusABG/raw/master/7-%20Acentuador.zip -O /tmp/Acentuador.zip
fi 
#unzip -p /tmp/Acentuador.zip "7- Acentuador/Corpus_Tag_Freq_Trans.txt" > /tmp/Corpus_Tag_Freq_Trans.txt
unzip -p /tmp/Acentuador.zip "7- Acentuador/Corpus_Transcrito.xlsx" > /tmp/Corpus_Transcrito.xlsx

Now we're going to convert the .xlsx file into a .csv file by using libreoffice.

In [33]:
libreoffice --headless --convert-to csv --outdir /tmp/ /tmp/Corpus_Transcrito.xlsx

convert /tmp/Corpus_Transcrito.xlsx -> /tmp/Corpus_Transcrito.csv using filter : Text - txt - csv (StarCalc)
Overwriting: /tmp/Corpus_Transcrito.csv


Lets check the type of file created.

In [34]:
file /tmp/Corpus_Transcrito.csv 

/tmp/Corpus_Transcrito.csv: Non-ISO extended-ASCII text


It is a problematic file. We're gonna assume it was coded in ISO 8859-1, which was standard for old Windows in Latin America and then convert it to UTF-8 using *translit* option (if can't find the right symbol in the target coding, chose the most visually similar).

In [35]:
iconv -f ISO-8859-1 -t UTF-8//TRANSLIT < /tmp/Corpus_Transcrito.csv  > /tmp/Corpus_Transcrito_utf8.csv

Lets create a list of all entries that still need a fix.

In [36]:
awk -F',[ ]*' 'BEGIN{OFS=","} ($2~/?/){print NR, $2, $4, $5, $9, $10, $11}' /tmp/CorpusABG.csv > /tmp/needfixlist

And not lets make a script to fix it.

For each entry that has ```?``` in it, which is listed in file ```/tmp/needfixlist``` created above, we will find the corresponding entry (same pronounciation) and replace the mispelled word by the vertion in the downloaded file. If we find the corresponding word, we will fix it and, if the lemma is equal to the word, we will also fix the lema.

In [38]:
rm -f /tmp/not_found.csv
while read line; do
  LINNUM=$(echo $line | awk -F',' '{print $1}')
  TRANSC=$(echo $line | awk -F',' '{print $4}')
  PWORD=$(echo $line | awk -F',' '{print $2}')
  #TRANSS="${TRANSC//[-&*]}"
  F1=$(echo $line | awk -F',' '{print $5}')
  F2=$(echo $line | awk -F',' '{print $6}')
  F3=$(echo $line | awk -F',' '{print $7}')
  WORD=$(awk -F'[ ]*,[ ]*' -v prn="$TRANSC" -v f1="$F1" -v f2="$F2" -v f3="$F3" 'BEGIN{OFS=","} $4==prn&&($8==f1||($8!~/^[0-9]+$/&&f1==0))&&($9==f2||($9!~/^[0-9]+$/&&f2==0))&&($10==f3||($10!~/^[0-9]+$/&&f3==0)){print $1}' /tmp/Corpus_Transcrito_utf8.csv)
  for word in $WORD;
  do
    if [ ${#word} == ${#PWORD} ];
      then
      THEWORD="$word"
    fi;
  done
  if [ ! -z "$THEWORD" ]; then
     awk -F',[ ]*' -v linnum="$LINNUM" -v word="$THEWORD" 'BEGIN{OFS=","} (NR==linnum&&$2==$4){$4=tolower(word)} NR==linnum{$2=tolower(word)} {$1=$1}1' /tmp/CorpusABG.csv > /tmp/tmpabg
     mv /tmp/tmpabg /tmp/CorpusABG.csv
  else
     echo "$PWORD, $TRANSC" >> /tmp/not_found.csv
  fi
  THEWORD=""
done < /tmp/needfixlist

Still there are many entries that still need fix.

In [39]:
echo $(wc -l /tmp/not_found.csv | cut -d ' ' -f1)"/"$(wc -l /tmp/needfixlist | cut -d ' ' -f1) | bc -l

.20504874063221953006


In [40]:
wc -l /tmp/not_found.csv 

3639 /tmp/not_found.csv


3639 entries (20.5%) were not fixed.

Lets see 10 random entries that were not fixed:

In [41]:
shuf -n 10 /tmp/not_found.csv

policial-rob?, &po-li-si-aW-ro-bo*
encoura?ado, &E-koW-ra-sa-do*
h?ngaros, &%-ga-ros*
circula??o, &sir-ku-la-sAW*
rememora??o, &he-me-mo-ra-sAW*
an?lises, &a-n1-li-es*
incipit?rio, &I-si-pi-t1-rJo*
DESIST?NCIA, &de-zis-tE-sJa*
manipul?-lo, &ma-ni-pu-l1-lo*
Execu??es, &e-ze-ku-sOJs*


In [43]:
# add number of phonemes and syllables
awk -F','  'BEGIN{OFS=","} NR==1{printf "%s",$0; printf "%s",",NUMFONES,NUMSILABAS\n"} NR>1{$2=tolower($2); $4=tolower($4); gsub(/^\&|\*$/,"",$5); gsub(/^\&|\*$/,"",$6); gsub(/^\&|\*$/,"",$7); printf "%s,",$0; gsub(/-/,"",$5); printf "%d,%d\n", length($5), gsub(/-/,"",$7)+1}' /tmp/CorpusABG.csv > /tmp/CorpusABGv2.csv

using a spell correction to correct entries with ```?```

In [ ]:
awk -F, 'BEGIN{OFS=FS} $2~/\?/{"echo \""$2"\" | ./spell.py"|& getline $2} $4~/\?/{"echo \""$4"\" | ./spell.py"|& getline $4} {$1=$1; print}' /tmp/CorpusABGv2.csv > /tmp/CorpusABGv2spell.csv

The substitutions made in field 2 are presented bellow:

In [44]:
diff <(awk -F, '{print $2}' /tmp/CorpusABGv2.csv) <(awk -F, '{print $2}' /tmp/CorpusABGv2spell.csv) | grep -v "^---" | grep -v "^[0-9c0-9]" | column 

< regi?o		< dic??o		< disposi??es
> região		> dicção		> disposições
< ningu?m		< flex?vel		< a?ucarado
> ninguém		> flexível		> açucarado
< qu?			< cal?ado		< s?c
> que			> calado		> sc
< opini?o		< desconstru??o		< rebeli?es
> opinião		> desconstrução		> rebeliões
< produ??o		< injusti?a		< radiois?topo
> produção		> injustiça		> radioisótopo
< s?rie			< sedu??o		< reconstr?i
> série			> sedução		> reconstrói
< servi?o		< viol?ncias		< brit?nicas
> serviço		> violências		> britânicas
< portugu?s		< destr?i		< desconex?o
> português		> destrói		> desconexo
< pr?prio		< descri??es		< comunica??o
> próprio		> descrições		> comunicação
< aten??o		< g?meos		< cal?ad?es
> atenção		> gêmeos		> calçadões
< condi??es		< escorpi?o		< injusti?ada
> condições		> escorpião		> injustiçada
< justi?a		< uni?es		< posti?os
> justiça		> uniões		> postiços
< constru??o		< extradi??o		< desperdi?a
> construção		> extradição		> desperdiça
< reuni?o		< redu??es		< degluti??o
> reunião		> reduções		> deglut

< cal?adas		< term?metros		< novi?a
> calçadas		> termômetros		> noviça
< prefer?ncias		< m?ller		< cadei?o
> preferências		> muller		> cadeião
< competi??es		< obsess?es		< sobreposi??es
> competições		> obsessões		> sobreposições
< resid?ncias		< mani?oba		< cr?nicos
> residências		> maniçoba		> crônicos
< licen?as		< mesti?agem		< realiza??es
> licenças		> mestiçagem		> realizações
< descal?o		< cal?amento		< prospec??o
> descalço		> calçamento		> prospecção
< inscri??es		< neurocirurgi?o	< transg?neros
> inscrições		> neurocirurgião	> transgêneros
< din?micas		< desservi?o		< hortali?as
> dinâmicas		> desserviço		> hortaliças
< dami?o		< antag?nicos		< redefini??o
> damião		> antagônicos		> redefinição
< v?nus			< desperdi?ou		< desuni?o
> vênus			> desperdiçou		> desunido
< influ?ncias		< pux?o			< predi?o
> influências		> puxão			> predigo
< obsess?o		< super-her?i		< dentu?o
> obsessão		> superherói		> dentuço
< caix?o		< descal?a		< coment?rios
> caixão		> descalça		> comentári

Unfortunately, some of them went wrong...

cal?ado > calado

s?c > sc

al?ar > aliar

lix?o > lixo

etc

There are many entries in the database which have at least one empty field.

there are still many entries which were not fixed

In [45]:
awk -F, '$2~/\?/{print $2}' /tmp/CorpusABGv2spell.csv | wc -l

3222


for example:

In [46]:
awk -F, '$2~/\?/{print $2}' /tmp/CorpusABGv2spell.csv | column | head

condi??o			beb?o
justi?a				?s
opini?o				reprercuss?o
uni?o				?ngulos
servi?os			sarad?es
atl?tico-mg			compreend?-los
produ??o			interse??es
servi?o				liter?rios
?				equalit?rio
introdu??o			colch?o


In [ ]:
awk -F',[ ]*' 'BEGIN{OFS=","} {test=0} {for (i = 1; i <= NF; i++) if ($i=="") test=1} (test){print}' /tmp/CorpusABG.csv | wc -l

In [ ]:
awk -F'[ ]*,[ ]*' 'BEGIN{OFS=","} {print NF}' /tmp/CorpusABG.csv | sort | uniq -c